In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [10]:

# seed 값 설정
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [11]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [12]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [15]:
img = load_img('../dataset/hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!!
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='../preview', 
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break

In [17]:
train_generator = train_datagen.flow_from_directory(
        '../dataset/hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '../dataset/hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [18]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

In [19]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

#### 모델 학습시키기
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 200번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [24]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15,#*100,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 1.2318e-06 - val_accuracy: 0.6667
Epoch 2/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0599 - accuracy: 0.9778 - val_loss: 0.0576 - val_accuracy: 0.8000
Epoch 3/200
15/15 [==============================] - 0s 10ms/step - loss: 0.1940 - accuracy: 0.9778 - val_loss: 5.9982 - val_accuracy: 0.8000
Epoch 4/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0284 - accuracy: 0.9778 - val_loss: 7.1703 - val_accuracy: 0.7333
Epoch 5/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0192 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 0.7333
Epoch 6/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 0.8000
Epoch 7/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0909 - accuracy: 0.9778 - val_loss: 8.7534 - val_accuracy: 0.733

Epoch 58/200
15/15 [==============================] - 0s 11ms/step - loss: 3.4190e-04 - accuracy: 1.0000 - val_loss: 5.6354 - val_accuracy: 0.8000
Epoch 59/200
15/15 [==============================] - 0s 11ms/step - loss: 8.0276e-04 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 0.7333
Epoch 60/200
15/15 [==============================] - 0s 11ms/step - loss: 0.1726 - accuracy: 0.9556 - val_loss: 0.8450 - val_accuracy: 0.6000
Epoch 61/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0147 - accuracy: 1.0000 - val_loss: 6.7093 - val_accuracy: 0.6000
Epoch 62/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 6.8342 - val_accuracy: 0.6000
Epoch 63/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.0212 - val_accuracy: 0.7333
Epoch 64/200
15/15 [==============================] - 0s 11ms/step - loss: 6.2952e-04 - accuracy: 1.0000 - val_loss: 10.1437 - val_acc

15/15 [==============================] - 0s 11ms/step - loss: 0.0934 - accuracy: 0.9778 - val_loss: 0.6955 - val_accuracy: 0.7333
Epoch 115/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 4.0184 - val_accuracy: 0.7333
Epoch 116/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0284 - accuracy: 0.9778 - val_loss: 1.4820 - val_accuracy: 0.7333
Epoch 117/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0307 - accuracy: 0.9778 - val_loss: 5.3713 - val_accuracy: 0.7333
Epoch 118/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 1.8332 - val_accuracy: 0.7333
Epoch 119/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.3829 - val_accuracy: 0.7333
Epoch 120/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 3.6804 - val_accuracy: 0.7333
Epoch 

Epoch 170/200
15/15 [==============================] - 0s 10ms/step - loss: 1.6955e-05 - accuracy: 1.0000 - val_loss: 10.9638 - val_accuracy: 0.6667
Epoch 171/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0334 - accuracy: 0.9778 - val_loss: 2.8124 - val_accuracy: 0.8000
Epoch 172/200
15/15 [==============================] - 0s 11ms/step - loss: 2.1926e-05 - accuracy: 1.0000 - val_loss: 3.4579 - val_accuracy: 0.8000
Epoch 173/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0439 - accuracy: 0.9778 - val_loss: 11.1695 - val_accuracy: 0.8000
Epoch 174/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 2.5269 - val_accuracy: 0.8000
Epoch 175/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.5895e-07 - val_accuracy: 0.8000
Epoch 176/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 6.2223 -

In [25]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 80.00%


In [23]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[0.000 0.000 1.000]
 [0.083 0.000 0.917]
 [0.000 0.968 0.032]
 [0.000 0.000 1.000]
 [0.000 0.999 0.001]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.126 0.874]
 [0.000 0.000 1.000]
 [0.000 1.000 0.000]
 [0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [1.000 0.000 0.000]]


In [ ]:

for i in range(len(test_generator.labels)):
    print(test_generator.filenames[i], test_generator.labels[i], np.argmax(output[i]))